In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
# tf.compat.v1.disable_eager_execution()


from Architectures.factorizer import Factorizer
from Architectures.ifn import mine_loss, joint, marginal, MI
from Architectures.dnn import DNN

# Data

In [2]:
x = np.expand_dims(np.random.binomial(1, 0.5, 10000), 1).astype(np.float32)
y = x

# Model

In [3]:
input_x = keras.layers.Input(shape=(1,))
h = keras.layers.Dense(16, activation='relu')(input_x)
h = keras.layers.Dense(16, activation='relu')(h)
output_x = keras.layers.Dense(1, activation='sigmoid')(h)
x_network_template = keras.Model(inputs=input_x, outputs=output_x)

input_y = keras.layers.Input(shape=(1,))
h = keras.layers.Dense(16, activation='relu')(input_y)
h = keras.layers.Dense(16, activation='relu')(h)
output_y = keras.layers.Dense(1, activation='sigmoid')(h)
y_network_template = keras.Model(inputs=input_y, outputs=output_y)




# x_network_template = DNN(1, [16, 16], 1, output_act = "sigmoid")
# y_network_template = DNN(1, [16, 16], 1, output_act = "sigmoid")


2022-03-03 16:17:36.409230: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
f1 = Factorizer(1, [DNN(1, (16, 16,), (1,), output_act="sigmoid"), ], [DNN(1, (16, 16,), (1,), output_act="sigmoid"), ])
f2 = Factorizer(2, x_network_template, y_network_template)

opt = tf.keras.optimizers.Adam(clipnorm = 1.0, learning_rate = 1e-3)
f1.compile(loss=mine_loss, optimizer=opt, metrics = [MI, joint, marginal])
f2.compile(loss=mine_loss, optimizer=opt, metrics = [MI, joint, marginal])

print(f2([x,y]))

[-1.45363688 -1.38629436 -1.45363688 ... -1.45363688 -1.38629436 -1.45363688]
tf.Tensor([-1.4536369 -1.3862944 -1.4536369 ... -1.4536369 -1.3862944 -1.4536369], shape=(10000,), dtype=float32)


# Fits

In [5]:
epochs = 50
batch_size = 512

history1 = f1.fit([x, y], batch_size= batch_size, epochs = epochs, shuffle=True, verbose= 2)
# history2 = f2.fit([x, y], batch_size= batch_size, epochs = epochs, shuffle=True, verbose= 2)

plt.plot(history1.history["MI"], label = "N = 1")
plt.plot(history2.history["MI"], label = "N = 2")
plt.plot(history2.history["MI"] * 0 + np.log(2.0), linestyle = "--", label = "Exact")
plt.xlabel("Epochs")
plt.ylabel("I(X;Y)")
plt.grid()
plt.legend()

2022-03-03 16:17:38.436323: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


TypeError: An op outside of the function building code is being passed
a "Graph" tensor. It is possible to have Graph tensors
leak out of the function building context by including a
tf.init_scope in your function building code.
For example, the following function will fail:
  @tf.function
  def has_init_scope():
    my_constant = tf.constant(1.)
    with tf.init_scope():
      added = my_constant * 2
The graph tensor has name: Reshape_2:0